# Convolutional experiments

The following experiments all use  one layer convnets in order to classify the sports ADL data set from UCI ML repo. Now maxpooling layer is used.

## No feature extraction regular convnets

The following 2 experiments  carry out convents. First with 5 feature maps out second with only 1. They both overfit massively in coparison to the regular convnets.

In [2]:
# Benchmark newest on 3 epochs

from mlp.convlin import *
import numpy
import logging
from mlp.dataset import *
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')
import numpy
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

from mlp.layers import MLP, Linear #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiserfrom mlp.dataset import MNISTDataProvider #import data provider
from mlp.costs import MSECost,CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed

rng = numpy.random.RandomState([2015,10,10])

train_dp = MACLDataProvider(dset='train', batch_size=100,
                            max_num_batches=-10, randomize=True, conv_reshape=True)
valid_dp = MACLDataProvider(dset='valid', batch_size=100,
                            max_num_batches=-10, randomize=False, conv_reshape=True)


cost = CECost()
model = MLP(cost=cost)
model.add_layer(ConvRelu_Opt(1, 5, rng=rng, stride=(1,1)))
model.add_layer(ConvMaxPool2D(5, (100,5, 122,122)))
# model.add_layer(ConvLinear_Opt(5, 10, rng=rng, image_shape=(12, 12) , kernel_shape=(5,5)))
# model.add_layer(ConvMaxPool2D(10, (100,10, 8,8), pool_shape=(2,2)))
# sigmoid 
model.add_layer(Sigmoid(idim=305, odim=121, rng=rng))

model.add_layer(Softmax(idim=121, odim=19, rng=rng))
# for sigmoid 0.1 is good for linear 0.005 and 0.1 is so so bigger not so good. ReLu : 0.1
lr_scheduler = LearningRateFixed(learning_rate=0.1, max_epochs=1000)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)
val, tra = optimiser.train(model, train_dp, valid_dp)
#tst_cost, tst_accuracy = optimiser.validate(model, valid_dp)

INFO:root:Initialising data providers...


(7296, 45, 125) (7296,)
(7296, 125)
(1824, 45, 125)

INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 3.023. Accuracy is 5.44%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 3.016. Accuracy is 5.11%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 2.870. Accuracy is 11.31%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 2.765. Accuracy is 20.67%
INFO:mlp.optimisers:Epoch 1: Took 1 seconds. Training speed 7743 pps. Validation speed 20011 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 2.627. Accuracy is 21.58%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 2.489. Accuracy is 25.00%
INFO:mlp.optimisers:Epoch 2: Took 1 seconds. Training speed 7501 pps. Validation speed 22513 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 2.421. Accuracy is 24.10%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 2.487. Accuracy is 22.17%
INFO:mlp.optimisers:Epoch 3: Took 1 seconds. Training speed 7743 pps. Validation speed 22513 pps.
INFO:mlp.optimisers:Epoch 4: Training cost (ce) 

 (1824,)
(1824, 125)


In [7]:
from mlp.layers import Relu
cost = CECost()
model = MLP(cost=cost)
model.add_layer(ConvRelu_Opt(1, 1, rng=rng, stride=(1,1)))
model.add_layer(ConvMaxPool2D(1, (100,1, 122,122)))
# model.add_layer(ConvLinear_Opt(5, 10, rng=rng, image_shape=(12, 12) , kernel_shape=(5,5)))
# model.add_layer(ConvMaxPool2D(10, (100,10, 8,8), pool_shape=(2,2)))
# sigmoid 
model.add_layer(Sigmoid(idim=61, odim=61, rng=rng))

model.add_layer(Softmax(idim=61, odim=19, rng=rng))
# for sigmoid 0.1 is good for linear 0.005 and 0.1 is so so bigger not so good. ReLu : 0.1
lr_scheduler = LearningRateFixed(learning_rate=0.1, max_epochs=1000)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)
val, tra = optimiser.train(model, train_dp, valid_dp)

INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.971. Accuracy is 5.38%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.975. Accuracy is 5.00%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 2.905. Accuracy is 8.96%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 2.862. Accuracy is 7.56%
INFO:mlp.optimisers:Epoch 1: Took 0 seconds. Training speed 25718 pps. Validation speed 45025 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 2.833. Accuracy is 11.35%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 2.812. Accuracy is 9.33%
INFO:mlp.optimisers:Epoch 2: Took 0 seconds. Training speed 25718 pps. Validation speed 60033 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 2.790. Accuracy is 11.75%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 2.773. Accuracy is 11.89%
INFO:mlp.optimisers:Epoch 3: Took 0 seconds. Training speed 25718 pps. Validation speed 60033 pps.
INFO:mlp.optimisers:Epoch 4: Training cost (ce) 

## Spectral input

the next layer uses $|\mathcal{F}\{x_{timeseries}\} |$ as input

In [15]:
# Benchmark newest on 3 epochs

from mlp.convlin import *
import numpy
import logging
from mlp.dataset import *
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')
import numpy
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

from mlp.layers import MLP, Linear #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiserfrom mlp.dataset import MNISTDataProvider #import data provider
from mlp.costs import MSECost,CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed

rng = numpy.random.RandomState([2015,10,10])

train_dp = MACLDataProvider(dset='train', batch_size=100, max_num_batches=-10, randomize=True, conv_reshape=True, fft=True)
valid_dp = MACLDataProvider(dset='valid', batch_size=100, max_num_batches=-10, randomize=False, conv_reshape=True, fft=True)


INFO:root:Initialising data providers...


(7296, 45, 125) (7296,)
(7296, 125)
(1824, 45, 125) (1824,)
(1824, 125)


In [20]:
from mlp.layers import Relu
cost = CECost()
model = MLP(cost=cost)
model.add_layer(ConvRelu_Opt(1, 1, rng=rng, stride=(1,1)))
model.add_layer(ConvMaxPool2D(1, (100,1, 122,122)))

# model.add_layer(ConvMaxPool2D(5, (100,5, 24,24)))
# model.add_layer(ConvLinear_Opt(5, 10, rng=rng, image_shape=(12, 12) , kernel_shape=(5,5)))
# model.add_layer(ConvMaxPool2D(10, (100,10, 8,8), pool_shape=(2,2)))
# sigmoid 
model.add_layer(Sigmoid(idim=61, odim=61, rng=rng))

model.add_layer(Softmax(idim=61, odim=19, rng=rng))
# for sigmoid 0.1 is good for linear 0.005 and 0.1 is so so bigger not so good. ReLu : 0.1
lr_scheduler = LearningRateFixed(learning_rate=0.09, max_epochs=1000)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)
val, tra = optimiser.train(model, train_dp, valid_dp)

INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.973. Accuracy is 5.17%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.968. Accuracy is 5.83%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 2.757. Accuracy is 17.12%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 2.504. Accuracy is 25.67%
INFO:mlp.optimisers:Epoch 1: Took 0 seconds. Training speed 30004 pps. Validation speed 90050 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 2.281. Accuracy is 35.74%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 2.160. Accuracy is 34.56%
INFO:mlp.optimisers:Epoch 2: Took 0 seconds. Training speed 30004 pps. Validation speed 90050 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 1.989. Accuracy is 40.82%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 1.939. Accuracy is 41.33%
INFO:mlp.optimisers:Epoch 3: Took 0 seconds. Training speed 28804 pps. Validation speed 90050 pps.
INFO:mlp.optimisers:Epoch 4: Training cost (c

# Two convnets 

In [23]:
from mlp.layers import Relu
cost = CECost()
model = MLP(cost=cost)
model.add_layer(ConvRelu_Opt(1, 1, rng=rng, stride=(1,1)))
model.add_layer(ConvMaxPool2D(1, (100,1, 122,122)))

model.add_layer(ConvRelu_Opt(1, 1, image_shape=(61, 61), rng=rng, stride=(1,1)))

model.add_layer(ConvMaxPool2D(1, (100,1, 58,58)))

model.add_layer(Sigmoid(idim=29, odim=29, rng=rng))

model.add_layer(Softmax(idim=29, odim=19, rng=rng))
# for sigmoid 0.1 is good for linear 0.005 and 0.1 is so so bigger not so good. ReLu : 0.1
lr_scheduler = LearningRateFixed(learning_rate=0.09, max_epochs=1000)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)
val, tra = optimiser.train(model, train_dp, valid_dp)

INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.952. Accuracy is 5.24%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.951. Accuracy is 5.17%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 2.904. Accuracy is 7.81%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 2.825. Accuracy is 14.61%
INFO:mlp.optimisers:Epoch 1: Took 0 seconds. Training speed 26670 pps. Validation speed 90050 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 2.693. Accuracy is 19.37%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 2.597. Accuracy is 22.00%
INFO:mlp.optimisers:Epoch 2: Took 0 seconds. Training speed 26670 pps. Validation speed 60033 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 2.476. Accuracy is 26.10%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 2.423. Accuracy is 26.33%
INFO:mlp.optimisers:Epoch 3: Took 0 seconds. Training speed 25718 pps. Validation speed 60033 pps.
INFO:mlp.optimisers:Epoch 4: Training cost (ce